In [ ]:
#Imports
!pip install requests
import requests
import pandas as pd
import json


In [ ]:
#@title **PVGIS data selection and parameters** { run: "auto", display-mode: "form" }
#@markdown ---
#@markdown To fetch the necessary data from [PVGIS](https://re.jrc.ec.europa.eu/pvg_tools/en/) it is necessary to select the following parameters:
#@markdown * Latitude
#@markdown * Longitude
#@markdown * Start Year - Year from which the simulation starts
#@markdown * End Year - Year in which the simulation ends
#@markdown * Optimal Azimuth - Year in which the simulation ends

#@markdown ![sad](https://upload.wikimedia.org/wikipedia/commons/f/f7/Azimuth-Altitude_schematic.svg)


#
#@markdown 
Latitude = 38.720 #@param {type:"number"}
Longitude = -9.281 #@param {type:"number"}

start_year = "2020" #@param ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]
end_year = "2020" #@param ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"]

#@markdown Consider the optimal azimuth for the specified location
optimal_azimuth = False #@param {type:"boolean"}
Slope = 90 #@param ["90"] {type:"raw"}


url = f" https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={Latitude}&lon={Longitude}&startyear={start_year}&endyear={end_year}&optimalangles={int(optimal_azimuth==True)}&angle={Slope}&outputformat=json"
#Gets the data from the url defined previously
api_response = requests.get(url)

#Converts into readable json
api_response_json = api_response.json()
data = json.loads(json.dumps(api_response_json))

azimuth_east = data['inputs']['mounting_system']['fixed']['azimuth']['value'] + 90
azimuth_west= data['inputs']['mounting_system']['fixed']['azimuth']['value'] - 90

url_east = f" https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={Latitude}&lon={Longitude}&startyear={start_year}&endyear={end_year}&aspect={azimuth_east}&angle={Slope}&outputformat=json"
url_west = f" https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={Latitude}&lon={Longitude}&startyear={start_year}&endyear={end_year}&aspect={azimuth_west}&angle={Slope}&outputformat=json"

api_response_east = requests.get(url_east)
api_response_json_east = api_response_east.json()
data_east = json.loads(json.dumps(api_response_json_east))
df_east = pd.DataFrame(data_east['outputs']['hourly'])
df_east.columns = ['Time', 'G(i)', 'H_sun', 'T2m', 'WS10m', 'Int']
df_east.set_index('Time', inplace=True)

api_response_west = requests.get(url_west)
api_response_json_west = api_response_west.json()
data_west = json.loads(json.dumps(api_response_json_west))
df_west = pd.DataFrame(data_west['outputs']['hourly'])
df_west.columns = ['Time', 'G(i)', 'H_sun', 'T2m', 'WS10m', 'Int']
df_west.set_index('Time', inplace=True)


df = df_east.add(df_west, fill_value=0)

